# Imaging with Stride

In this notebook, we will apply what we have seen previously to image a 2D section of the human breast using full-waveform inversion (FWI) in Stride.

FWI is a tomographic imaging technique, originally developed in geophysics, that solves a local optimisation problem in order to find the acoustic properties (usually speed of sound) of some imaged region. 

Given some data that we have collected or observed $d_{obs}$ and some data that we have modelled numerically $d_{mod}$, FWI solves the problem,

$$
v_p^* = argmin_{v_p} J(v_p, d_{obs}, d_{mod})
$$
$$
s.t.\; \mathbf{L}(d_{mod}, v_p) = \mathbf{0}
$$

to find the speed of sound $v_p$, where $J(v_p, d_{obs}, d_{mod})$ is a scalar loss function and $\mathbf{L}(d_{mod},v_p) = \mathbf{0}$ is a system of PDEs.

Most generally, we use the L2-norm of the distance between $d_{obs}$ and $d_{mod}$ as the loss function, and the acoustic wave equation as the PDE. The problem then becomes,

$$
v_p^* = argmin_{v_p} \left\langle \frac{1}{2} \left( p(v_p) \delta(\mathbf{x}-\mathbf{x}_0) - d_{obs} \right) ^2, 1 \right\rangle
$$
$$
s.t.\; \frac{1}{v_p^2} \frac{\partial^2 p}{\partial t^2} - \nabla^2 p - s = 0
$$

where $\left\langle \alpha, \beta \right\rangle = \int_{t_0}^{t_1}\int_{\Omega} \alpha \cdot \beta d\mathbf{x}^3dt$ for $t \in [t_0, t_1]$ and $\mathbf{x} \in \Omega$, $\delta(x)$ is the sampling Dirac delta, $\mathbf{x}_0$ is the location of receivers in the region, $p$ is the pressure, and $s$ is a source term.

In FWI, we solve this problem by using a local optimisation method like gradient descent, and we calculate the gradient of the problem using the adjoint method or, what is the same, the procedure that we introduced in the first notebook of this tutorial.

## Mosaic runtime

Before we proceed, let's start the Mosaic runtime.

In [ ]:
import mosaic
await mosaic.interactive('off')
await mosaic.interactive('on', num_workers=2, log_level='info')
runtime = mosaic.runtime()

## Problem definition

Let's start by defining out problem as usual, starting with the spatiotemporal grid.

In [ ]:
from stride import Space, Time, Grid
%matplotlib widget

space = Space(shape=(356, 385), extra=(50, 50), absorbing=(40, 40), spacing=0.5e-3)
time = Time(start=0.0e-6, step=0.08e-6, num=2500)

grid = Grid(space, time)

We create the problem object.

In [ ]:
from stride import Problem

problem = Problem(name='breast2D', space=space, time=time)

And we start filling it up we our transducers and geometry.

In [ ]:
# Create transducers
# the default option will create a single point transducer
problem.transducers.default()

# Create geometry
# a default elliptical geometry will be generated in this case
num_locations = 120
problem.geometry.default('elliptical', num_locations)

We are going to populate our acquisition sequence with 120 shots (one for each location in the geoemtry). During each shot, a different location is going to act as the source and the rest of them are going to act as receivers.

We can easily do this in Stride by calling the `default()` method of the `stride.Acquisitions` object.

In [ ]:
from stride.utils import wavelets

# Populate acquisitions with default shots
problem.acquisitions.default()

# Create wavelets
f_centre = 0.50e6
n_cycles = 3

for shot in problem.acquisitions.shots:
    shot.wavelets.data[0, :] = wavelets.tone_burst(f_centre, n_cycles, time.num, time.step)

Now, let's load our breast model.

In [ ]:
from stride import ScalarField

# Create medium
vp_true = ScalarField(name='vp', grid=grid)
vp_true.load('../examples/breast2D/data/anastasio2D-TrueModel.h5')

problem.medium.add(vp_true)
problem.plot()


In [ ]:
wave_lambda = 1500. / f_centre
sigma = wave_lambda / 4
for loc in problem.geometry.locations:
    x , y = loc.coordinates 
    perturb_x = random.normal(0,sigma)
    perturb_y = random.normal(0,sigma)
    loc.coordinates  = np.array([x+perturb_x , y +perturb_y])
problem.plot()

## Modelling operator

Once we have generated our problem, we can create a PDE on which to run our forward problem. In this case, we choose the isotropic acoustic wave equation.

We instantiate our PDE using `remote(len=<num>)` so that instances of the PDE are created in the workers for remote execution. We choose `len=runtime.num_workers` so that one instance is created in each available worker.

In [ ]:
from stride import IsoAcousticDevito

pde = IsoAcousticDevito.remote(grid=problem.grid, len=runtime.num_workers)

## Forward problem - Generating observed data

Before we can proceed with the imaging bit, we need to simulate the acquisition of some observed data.

Here, we will exploit the parallelisation capabilities of Mosaic to run different shots in parallel in each available worker. We do this with an asynchronous for loop using `@runtime.async_for(<iterable>)`. In the previous code section, the async function `loop()` is called as many times as there are shots in the acquisitions and, for each shot, it is assigned a worker.

In [ ]:
# Get all remaining shot IDs
shot_ids = problem.acquisitions.remaining_shot_ids

# Run an asynchronous loop across all shot IDs
@runtime.async_for(shot_ids)
async def loop(worker, shot_id):
    runtime.logger.info('Giving shot %d to %s' % (shot_id, worker.uid))

    # Fetch one sub-problem corresponding to a shot ID
    sub_problem = problem.sub_problem(shot_id)
    
    # Access the source wavelets of this shot
    wavelets = sub_problem.shot.wavelets
    
    # Execute the PDE forward
    traces = await pde(wavelets, vp_true,
                       problem=sub_problem,
                       runtime=worker).result()

    runtime.logger.info('Shot %d retrieved' % sub_problem.shot_id)

    # Store the retrieved traces into the shot
    shot = problem.acquisitions.get(shot_id)
    shot.observed.data[:] = traces.data

    runtime.logger.info('Retrieved traces for shot %d' % sub_problem.shot_id)

# Because this is an asynchronous loop, it needs to be awaited 
_ = await loop

# Plot the result
_ = problem.acquisitions.plot()

Because the loop we just ran is a very common piece of code, we can obtain the same result by running the utility function `forward()`:

In [ ]:
from stride import forward

# Run default forward workflow
# await forward(problem, pde, vp_true, dump=False)  # uncomment to run using utility function


In [ ]:
import numpy.random as random
wave_lambda = 1500. / f_centre
sigma = wave_lambda / 4

print(problem.geometry.locations[0].coordinates)

for loc in problem.geometry.locations:
    x , y = loc.coordinates 
    perturb_x = random.normal(0,sigma)
    perturb_y = random.normal(0,sigma)
    loc.coordinates  = np.array([x+perturb_x , y +perturb_y])
    
print(problem.geometry.locations[0].coordinates)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
sub = problem.acquisitions.get(2)
signal = sub.observed.data[2]
# sub.observed.data[1] = np.random.rand(2500)

noise = np.random.normal(0, 0.2, signal.shape)
noisy_signal = signal + noise

# Step 3: Visualization
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(signal[:300], label='Original Signal')
plt.title('Original Signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(noise[:300], label='Noisy Signal')
plt.title('Signal with Noise')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid(True)

plt.tight_layout()
plt.show()

# Step 4: Compute Signal-to-Noise Ratio (SNR)
SP = np.sqrt(np.mean(signal[:300] ** 2))
NP = np.sqrt(np.mean(noise[:300] ** 2))
SNR = 10 * np.log10(SP / NP)
print(f'Signal-to-Noise Ratio (SNR): {SNR:.2f} dB')

## Starting model

Before we can proceed with the imaging process, we need to determine a starting point for our inversion, a starting model.

In [ ]:
import math
import random
import numpy as np

vp = ScalarField.parameter(name='vp', grid=grid, needs_grad=True)
vp.fill(1500.)

problem.medium.add(vp)

wave_lambda = 1500. / f_centre
perturb = wave_lambda / 40

class Coords:
    def __init__(self,coordinates):
        self.coordinates =coordinates
    
    def load(self):
        return self.coordinates
    
class Description:
    def __init__(self):
        self.locations = []

    def add_item(self, id ,transducer_id,coordinates):
        location_desc = LocationDescription(id, transducer_id, coordinates)
        self.locations.append(location_desc)

class LocationDescription:
    def __init__(self, id, transducer_id, coordinates, orientation=None):
        self.id = id
        self.transducer_id = transducer_id
        self.coordinates = Coords(coordinates)
        self.orientation = orientation

    def __iter__(self):
    # Yielding attributes one by one
        yield ('id', self.id)
        yield ('transducer_id', self.transducer_id)
        yield ('coordinates', self.coordinates)
        if self.orientation is not None:
            yield ('orientation', self.orientation)

# sub_problem = problem.sub_problem(shot.id)
descript= problem.geometry.__get_desc__()
print("original desc:", descript["locations"])
new_desc = Description()

for loc in descript["locations"]:
    
    original_x, original_y = loc["coordinates"]
    angle = random.uniform(0, 2 * math.pi)
    new_x = original_x + perturb * math.cos(angle)
    new_y = original_y + perturb * math.sin(angle)
    loc["coordinates"] = [new_x, new_y]

    new_desc.add_item(loc["id"], loc["transducer_id"], np.array(loc["coordinates"]))
    # print("Updated coordinates:", loc["coordinates"])

problem.geometry.__set_desc__(new_desc)
descript= problem.geometry.__get_desc__()
print("new desc:", descript["locations"])

    

Unlike our forward example, the speed-of-sound field is unknown to us at this point. So, we define our starting guess for the speed of sound to be homogeneous with a value of 1500 m/s. Because we are going to run our inversion with respect to `vp`, we also define the field with `needs_grad=True`. That will instruct Stride to calculate the gradient of this variable when running the optimisation loop.

You can also see that the field has now been instantiated using `parameter()`. This will turn `vp` into a remotely addressable object. What this means is that, as `vp` travels through our Mosaic network, it will always keep a reference to its original object here in our code. This will effectively allow us to accumulate the gradients calculated across different workers into a single local buffer.

## Imaging operators

Apart from our PDE operator, which we have already defined above, we will need to define an operator for our loss function:

In [ ]:
from stride import L2DistanceLoss 

loss = L2DistanceLoss.remote(len=runtime.num_workers)

We will also need some operators to proprocess the source wavelents and the modelled and observed data traces:

In [ ]:
from stride import ProcessWavelets, ProcessObserved, ProcessWaveletsObserved, ProcessTraces

process_wavelets = ProcessWavelets.remote(len=runtime.num_workers)
process_observed = ProcessObserved.remote(len=runtime.num_workers)
process_wavelets_observed = ProcessWaveletsObserved.remote(len=runtime.num_workers)
process_traces = ProcessTraces.remote(len=runtime.num_workers)

Finally, we will need an optimiser to update the speed of sound model after each iteration. In this case, we use gradient descent.

In [ ]:
from stride import GradientDescent, ProcessGlobalGradient, ProcessModelIteration

step_size = 10
process_grad = ProcessGlobalGradient()
process_model = ProcessModelIteration(min=1400., max=1700.)

optimiser = GradientDescent(vp, step_size=step_size,
                            process_grad=process_grad,
                            process_model=process_model)

You can see that we have also provided to the optimiser some pre-defined processing steps. These will be used to prepare the gradient before updating the variable, and to process the variable after the update. 

Among other things, these processing steps will normalise and smooth the gradient, and will clip the speed of sound of the updated model between 1400 m/s and 1700 m/s.

## Inverse problem - Estimating speed of sound

We can now proceed to solve the inverse problem, that is finding the speed of sound that explains the data that we have generated above.

To obtain a better posed optimisation, we use a multi-scale approach. We will start our inversion by using only low frequencies to construct our model. As the inversion progresses, we will add higher frequencies until we reach the maximum desired level of detail.

We can do this by dividing our optimisation in `Block`s and specifying a maximum frequency. Each `Block` will run for a number of specified iterations.

During each iteration, only a subset of the shots will be used by defining the `select_shots` field. In this case, 15 shots will be used for each iteration, and they will be selected randomly without replacement.

In [ ]:
from stride import OptimisationLoop

# Clear the previous Devito operators
await pde.clear_operators()

optimisation_loop = OptimisationLoop()

# Specify a series of frequency bands, which we will introduce gradually 
# into the inversion in order to better condition it
max_freqs = [0.3e6, 0.4e6]

num_blocks = len(max_freqs)
num_iters = 4

# Start iterating over each block in the optimisation
for block, f_max in optimisation_loop.blocks(num_blocks, max_freqs):

    # Proceed through every iteration in the block
    for iteration in block.iterations(num_iters):
        runtime.logger.info('Starting iteration %d (out of %d), '
                            'block %d (out of %d)' %
                            (iteration.id+1, block.num_iterations, block.id+1,
                             optimisation_loop.num_blocks))

        # Select some shots for this iteration
        shot_ids = problem.acquisitions.select_shot_ids(num=15, randomly=True)

        # Clear the gradient buffers of the variable
        vp.clear_grad()

        # Asynchronously loop over all the selected shot IDs
        @runtime.async_for(shot_ids)
        async def loop(worker, shot_id):
            runtime.logger.info('Giving shot %d to %s' % (shot_id, worker.uid))

            # Fetch one sub-problem corresponding to the shot ID
            sub_problem = problem.sub_problem(shot_id)
            wavelets = sub_problem.shot.wavelets
            observed = sub_problem.shot.observed

            # Pre-process the wavelets and observed
            wavelets = process_wavelets(wavelets, f_max=f_max, filter_relaxation=0.75, runtime=worker)
            observed = process_observed(observed, f_max=f_max, filter_relaxation=0.75, runtime=worker)
            processed = process_wavelets_observed(wavelets, observed, f_max=f_max, runtime=worker)
            wavelets = processed.outputs[0]
            observed = processed.outputs[1]
            
            # Execute the PDE forward
            modelled = pde(wavelets, vp, problem=sub_problem, runtime=worker)

            # Pre-process the modelled and observed traces
            traces = process_traces(modelled, observed, f_max=f_max, filter_relaxation=0.75, runtime=worker)
            # and use these pre-processed versions to calculate the
            # value of the loss_freq function
            fun = await loss(traces.outputs[0], traces.outputs[1],
                             problem=sub_problem, runtime=worker).result()

            iteration.add_loss(fun)
            runtime.logger.info('Functional value for shot %d: %s' % (shot_id, fun))

            # Now, we can calculate the gradient by executing the adjoint of the
            # forward process
            await fun.adjoint()

            runtime.logger.info('Retrieved gradient for shot %d' % sub_problem.shot_id)

        # Because this is an async loop, it needs to be awaited    
        _ = await loop
        # Update the vp with the calculated gradient by taking a step with the optimiser
        await optimiser.step()

        runtime.logger.info('Done iteration %d (out of %d), '
                            'block %d (out of %d) - Total loss_freq %e' %
                            (iteration.id+1, block.num_iterations, block.id+1,
                             optimisation_loop.num_blocks, iteration.total_loss))
        runtime.logger.info('====================================================================')

# Plot the vp afterwards   
vp.plot()

As was the case for the forward example, we can obtain the same result by using a utility function that will run the default inversion workflow using `adjoint`:

In [ ]:
import math
import random
import numpy as np
wave_lambda = 1500. / f_centre
perturb = wave_lambda / 4

class Coords:
    def __init__(self,coordinates):
        self.coordinates =coordinates
    
    def load(self):
        return self.coordinates
    
class Description:
    def __init__(self):
        self.locations = []

    def add_item(self, id ,transducer_id,coordinates):
        location_desc = LocationDescription(id, transducer_id, coordinates)
        self.locations.append(location_desc)

class LocationDescription:
    def __init__(self, id, transducer_id, coordinates, orientation=None):
        self.id = id
        self.transducer_id = transducer_id
        self.coordinates = Coords(coordinates)
        self.orientation = orientation

    def __iter__(self):
    # Yielding attributes one by one
        yield ('id', self.id)
        yield ('transducer_id', self.transducer_id)
        yield ('coordinates', self.coordinates)
        if self.orientation is not None:
            yield ('orientation', self.orientation)

# sub_problem = problem.sub_problem(shot.id)
descript= problem.geometry.__get_desc__()
print("original desc:", descript["locations"])
new_desc = Description()

for loc in descript["locations"]:
    
    original_x, original_y = loc["coordinates"]
    angle = random.uniform(0, 2 * math.pi)
    new_x = original_x + perturb * math.cos(angle)
    new_y = original_y + perturb * math.sin(angle)
    loc["coordinates"] = [new_x, new_y]

    new_desc.add_item(loc["id"], loc["transducer_id"], np.array(loc["coordinates"]))
    # print("Updated coordinates:", loc["coordinates"])

problem.geometry.__set_desc__(new_desc)
descript= problem.geometry.__get_desc__()
print("new desc:", descript["locations"])

    

### run reverse again with shift

In [ ]:
from stride import OptimisationLoop
vp2 = ScalarField.parameter(name='vp', grid=grid, needs_grad=True)
vp2.fill(1500.)

problem.medium.add(vp2)
# Clear the previous Devito operators
await pde.clear_operators()

optimisation_loop = OptimisationLoop()

# Specify a series of frequency bands, which we will introduce gradually 
# into the inversion in order to better condition it
max_freqs = [0.3e6, 0.4e6]

num_blocks = len(max_freqs)
num_iters = 4

# Start iterating over each block in the optimisation
for block, f_max in optimisation_loop.blocks(num_blocks, max_freqs):

    # Proceed through every iteration in the block
    for iteration in block.iterations(num_iters):
        runtime.logger.info('Starting iteration %d (out of %d), '
                            'block %d (out of %d)' %
                            (iteration.id+1, block.num_iterations, block.id+1,
                             optimisation_loop.num_blocks))

        # Select some shots for this iteration
        shot_ids = problem.acquisitions.select_shot_ids(num=15, randomly=True)

        # Clear the gradient buffers of the variable
        vp2.clear_grad()

        # Asynchronously loop over all the selected shot IDs
        @runtime.async_for(shot_ids)
        async def loop(worker, shot_id):
            runtime.logger.info('Giving shot %d to %s' % (shot_id, worker.uid))

            # Fetch one sub-problem corresponding to the shot ID
            sub_problem = problem.sub_problem(shot_id)
            wavelets = sub_problem.shot.wavelets
            observed = sub_problem.shot.observed

            # Pre-process the wavelets and observed
            wavelets = process_wavelets(wavelets, f_max=f_max, filter_relaxation=0.75, runtime=worker)
            observed = process_observed(observed, f_max=f_max, filter_relaxation=0.75, runtime=worker)
            processed = process_wavelets_observed(wavelets, observed, f_max=f_max, runtime=worker)
            wavelets = processed.outputs[0]
            observed = processed.outputs[1]
            
            # Execute the PDE forward
            modelled = pde(wavelets, vp2, problem=sub_problem, runtime=worker)

            # Pre-process the modelled and observed traces
            traces = process_traces(modelled, observed, f_max=f_max, filter_relaxation=0.75, runtime=worker)
            # and use these pre-processed versions to calculate the
            # value of the loss_freq function
            fun = await loss(traces.outputs[0], traces.outputs[1],
                             problem=sub_problem, runtime=worker).result()

            iteration.add_loss(fun)
            runtime.logger.info('Functional value for shot %d: %s' % (shot_id, fun))

            # Now, we can calculate the gradient by executing the adjoint of the
            # forward process
            await fun.adjoint()

            runtime.logger.info('Retrieved gradient for shot %d' % sub_problem.shot_id)

        # Because this is an async loop, it needs to be awaited    
        _ = await loop
        # Update the vp with the calculated gradient by taking a step with the optimiser
        await optimiser.step()

        runtime.logger.info('Done iteration %d (out of %d), '
                            'block %d (out of %d) - Total loss_freq %e' %
                            (iteration.id+1, block.num_iterations, block.id+1,
                             optimisation_loop.num_blocks, iteration.total_loss))
        runtime.logger.info('====================================================================')

# Plot the vp afterwards   
vp2.plot()

In [ ]:
print(problem.acquisitions.shot_ids)
subproblem = problem.sub_problem(10)
print(subproblem.shot.observed.data)

In [ ]:
await mosaic.interactive('off')